# Generate a subgraph covering CbGpPWpGaD paths between bupropion and nicotine dependence

In [1]:
import hetio.readwrite
import hetio.pathtools
import hetio.stats

In [2]:
# Read Hetionet v1.0
url = 'https://github.com/dhimmel/hetionet/raw/{}/{}'.format(
    '00bf0b6f8886821d91cfdf00eadad145a7a1b6da',
    'hetnet/json/hetionet-v1.0.json.bz2',
)
graph = hetio.readwrite.read_graph(url)
metagraph = graph.metagraph

In [3]:
# Specify compound and disease
compound_id = 'Compound', 'DB01156'  # Bupropion
disease_id = 'Disease', 'DOID:0050742'  # nicotine dependences
metapath = metagraph.metapath_from_abbrev('CbGpPWpGaD')

In [4]:
# Extract walks from compound to disease
paths = hetio.pathtools.paths_between(
    graph, 
    source=graph.node_dict[compound_id],
    target=graph.node_dict[disease_id],
    metapath=metapath,
    duplicates=True,
)

In [5]:
metaedges = set(metapath)
nodes = set()
for path in paths:
    nodes.update(path.get_nodes())
    for edge in path:
        # Add incidental nodes along paths to enable correct DWWC/DWPC computations
        # for the CbGpPWpGaD metapath
        nodes.update(e.target for e in edge.source.get_edges(edge.metaedge))
        nodes.update(e.target for e in edge.target.get_edges(edge.metaedge.inverse))

# Add Gene-interacts-Gene metaedge (not essential but may be useful)
metaedges.add(metagraph.metapath_from_abbrev('GiG')[0])

# Get subgraph
subgraph = graph.get_subgraph(metaedges=metaedges, nodes=nodes)

In [6]:
# Metagraph size
subgraph.metagraph.n_nodes, subgraph.metagraph.n_edges

(4, 4)

In [7]:
# Graph size
subgraph.n_nodes, subgraph.n_edges

(3060, 18474)

In [8]:
# Metanode info
hetio.stats.get_metanode_df(subgraph)

,metanode,abbreviation,metaedges,nodes,unconnected_nodes
0,Compound,C,1,730,730
3,Disease,D,1,16,16
1,Gene,G,4,2226,612
2,Pathway,PW,1,88,0


In [9]:
# Metaedge info
hetio.stats.get_metaedge_df(subgraph)

,metaedge,abbreviation,edges,source_nodes,target_nodes
0,Compound - binds - Gene,CbG,5634,730,472
1,Disease - associates - Gene,DaG,937,16,557
2,Gene - interacts - Gene,GiG,5124,1061,1213
3,Gene - participates - Pathway,GpPW,6779,2222,88


In [10]:
# Export as JSON
hetio.readwrite.write_graph(subgraph, 'bupropion-CbGpPWpGaD-subgraph.json.xz')